In [1]:
using DelimitedFiles
using Statistics
using DataFrames
using CSV
using Printf
using DataStructures
using Base.Threads
using ProgressMeter
using NPZ
using ArgParse
using JSON
using Dates
using ArgParse
using HDF5
#using NCDatasets
using NetCDF

In [2]:
base_folder = ".."

include(joinpath(base_folder, "MMCAcovid19_vac/markov_vac_aux.jl"))
include(joinpath(base_folder, "MMCAcovid19_vac/markov_vac.jl"))

run_epidemic_spreading_mmca!

FUNCTIONS

In [3]:
function set_compartments!(epi_params, initial_compartments)
    @assert size(initial_compartments) == (size(epi_params.ρˢᵍᵥ)[1], size(epi_params.ρˢᵍᵥ)[2], size(epi_params.ρˢᵍᵥ)[4], 10)
    total_population = sum(initial_compartments, dims=(3))[:,:,1]
    
    # Index of the initial condition
    T0 = 1
    
    epi_params.ρˢᵍᵥ[:,:,T0,:]  .= initial_compartments[:, :, T0, :, 1] ./ total_population
    epi_params.ρᴱᵍᵥ[:,:,T0,:]  .= initial_compartments[:, :, T0, :, 2] ./ total_population
    epi_params.ρᴬᵍᵥ[:,:,T0,:]  .= initial_compartments[:, :, T0, :, 3] ./ total_population
    epi_params.ρᴵᵍᵥ[:,:,T0,:]  .= initial_compartments[:, :, T0, :, 4] ./ total_population
    epi_params.ρᴾᴴᵍᵥ[:,:,T0,:] .= initial_compartments[:, :, T0, :, 5] ./ total_population
    epi_params.ρᴾᴰᵍᵥ[:,:,T0,:] .= initial_compartments[:, :, T0, :, 6] ./ total_population
    epi_params.ρᴴᴿᵍᵥ[:,:,T0,:] .= initial_compartments[:, :, T0, :, 7] ./ total_population
    epi_params.ρᴴᴰᵍᵥ[:,:,T0,:] .= initial_compartments[:, :, T0, :, 8] ./ total_population
    epi_params.ρᴿᵍᵥ[:,:,T0,:]  .= initial_compartments[:, :, T0, :, 9] ./ total_population
    epi_params.ρᴰᵍᵥ[:,:,T0,:]  .= initial_compartments[:, :, T0, :, 10] ./ total_population

    epi_params.ρˢᵍᵥ[isnan.(epi_params.ρˢᵍᵥ)] .= 0
    epi_params.ρᴱᵍᵥ[isnan.(epi_params.ρᴱᵍᵥ)] .= 0
    epi_params.ρᴬᵍᵥ[isnan.(epi_params.ρᴬᵍᵥ)] .= 0
    epi_params.ρᴵᵍᵥ[isnan.(epi_params.ρᴵᵍᵥ)] .= 0
    epi_params.ρᴾᴴᵍᵥ[isnan.(epi_params.ρᴾᴴᵍᵥ)] .= 0
    epi_params.ρᴾᴰᵍᵥ[isnan.(epi_params.ρᴾᴰᵍᵥ)] .= 0
    epi_params.ρᴴᴿᵍᵥ[isnan.(epi_params.ρᴴᴿᵍᵥ)] .= 0
    epi_params.ρᴴᴰᵍᵥ[isnan.(epi_params.ρᴴᴰᵍᵥ)] .= 0
    epi_params.ρᴿᵍᵥ[isnan.(epi_params.ρᴿᵍᵥ)] .= 0
    epi_params.ρᴰᵍᵥ[isnan.(epi_params.ρᴰᵍᵥ)] .= 0
end

set_compartments! (generic function with 1 method)

In [4]:
function run_simu_params!(epi_params::Epidemic_Params,
                          population::Population_Params,
                          paramsDF::DataFrame,
                          indx_id::Int64,
                          E₀::Array{Float64, 2},
                          A₀::Array{Float64, 2},
                          I₀::Array{Float64, 2},
                          H₀::Array{Float64, 2},
                          R₀::Array{Float64, 2},
                          Sᵛ₀::Array{Float64, 2},
                          compartments::Union{Nothing, Array{Float64, 6}})

    # Loading parameters from DF
    id, P, β, scale_β, τ_inc, scale_ea, τᵢ, δ, ϕ, scale₀ = paramsDF[indx_id, :]

    # Set epidemic params to the ones speficied on the DF
    epi_params.βᴬ .= scale_β * β
    epi_params.βᴵ .= β
    epi_params.ηᵍ .= 1.0/(τ_inc * (1.0 - scale_ea))
    epi_params.αᵍ .= [1.0/(τᵢ - 1 + τ_inc * scale_ea),
                      1.0/(τ_inc * scale_ea),
                      1.0/(τ_inc * scale_ea)]
    epi_params.μᵍ .= [1.0, 1.0/τᵢ, 1.0/τᵢ]


    # Rescaling the initial number of asymptomatic
    scaled_A₀ = scale₀ .* A₀

    #descomentado
    # Set containment parameters
    ϕs .= ϕ
    δs .= δ

    # Reset compartments
    reset_params!(epi_params, population)
    
    if initial_compartments != nothing
        set_compartments!(epi_params, initial_compartments)
    else
        set_initial_conditions!(epi_params, population, Sᵛ₀, E₀, scaled_A₀, I₀, H₀, R₀)
    end

    
    ## RUN EPIDEMIC SPREADING
    run_epidemic_spreading_mmca!(epi_params, population, tᶜs, tᵛs, κ₀s, ϕs, δs, ϵᵍs; verbose = true )
    
    # Store compartments to later export (can't write to disk here, hdf5 is not thread safe)
    if export_compartments
        compartments[:, :, :, :, 1, indx_id]  .= epi_params.ρˢᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 2, indx_id]  .= epi_params.ρᴱᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 3, indx_id]  .= epi_params.ρᴬᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 4, indx_id]  .= epi_params.ρᴵᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 5, indx_id]  .= epi_params.ρᴾᴴᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 6, indx_id]  .= epi_params.ρᴾᴰᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 7, indx_id]  .= epi_params.ρᴴᴿᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 8, indx_id]  .= epi_params.ρᴴᴰᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 9, indx_id]  .= epi_params.ρᴿᵍᵥ .* population.nᵢᵍ
        compartments[:, :, :, :, 10, indx_id] .= epi_params.ρᴰᵍᵥ .* population.nᵢᵍ
    end
    # PIER: if we want to save just the age, location and time infos we have to use
    # compartments[:, :, :, :, 1, indx_id] .= sum( epi_params.ρˢᵍᵥ .* population.nᵢᵍ, dims=(4) )[:,:,:,1]
end

run_simu_params! (generic function with 1 method)

In [5]:
data_path = joinpath(base_folder,"data/")
instance_path = joinpath(base_folder,"test")

# Output simulation
output_path = joinpath(instance_path, "output")


config_fname = joinpath(instance_path, "config.json")
config = JSON.parsefile(config_fname);

fitting_params_fname = joinpath(instance_path, "fitting_parameters.json")
fitting_params = JSON.parsefile(fitting_params_fname)

vacparams_dict = config["vaccination"]
npiparams_dict = config["NPI"]
epiparams_dict = config["model"]

first_day = Date(config["simulation"]["first_day_simulation"])
last_day = Date(config["simulation"]["last_day_simulation"])



T = (last_day - first_day).value + 1

A0_instance_filename = get(config["simulation"], "A0_filename", nothing)
A0_instance_filename = joinpath(instance_path, A0_instance_filename)

initial_compartments_path = get(config["simulation"], "initial_compartments", nothing)

if A0_instance_filename !== nothing && initial_compartments_path !== nothing
    println("ERROR!!!")
end

SIMULATION OUTPUT

In [6]:
export_compartments_full = get(config["simulation"], "export_compartments_full", false)
export_compartments_time_t = get(config["simulation"], "export_compartments_time_t", nothing)

if export_compartments_time_t != nothing
    export_compartments_date = first_day + Day(export_compartments_time_t - 1)
end

if export_compartments_full || export_compartments_time_t != nothing
    export_compartments = true
else
    export_compartments = false
end

println("first_day_simulation = ", first_day)
println("last_day_simulation = ", last_day)
println("export_compartments = ", export_compartments)
println("export_compartments_full = ", export_compartments_full)
println("export_compartments_time_t = ", export_compartments_time_t)
println("initial_compartments = ", initial_compartments_path)

num_compartments = 10

first_day_simulation = 2020-02-09


last_day_simulation = 2020-03-09
export_compartments = true
export_compartments_full = true


export_compartments_time_t = nothing
initial_compartments = nothing


10

In [7]:
# Patch surface
sᵢ = CSV.read(joinpath(data_path, config["data"]["surface_filename"]), DataFrame)[:,"area"]
# Population info
nᵢ_ages = CSV.read(joinpath(data_path, config["data"]["population_age_filename"], ), DataFrame);
# Patch population by age
nᵢᵍ = copy(transpose(Array{Float64,2}(nᵢ_ages[:,epiparams_dict["age_labels"]])))
# Total patch population
nᵢ = Array{Float64,1}(nᵢ_ages[:,"Total"])
# Total population
total_population = sum(nᵢ)

# Age Contact Matrix
C = readdlm(joinpath(data_path, config["data"]["contact_matrix_filename"]), ',', Float64)
# Num. of patches
M = length(nᵢ)
# Num of stratas
G = size(C)[1]
# Num. of vaccination statuses Vaccinated/Non-vaccinated
V = length(epiparams_dict["kᵥ"])


# Loading mobility network
network = CSV.read(joinpath(data_path, config["data"]["mobility_matrix_filename"]), DataFrame)
edgelist = Array{Int64, 2}(network[:, 1:2])
Rᵢⱼ = copy(network[:, 3])
# Correcting Self Loops
edgelist, Rᵢⱼ = correct_self_loops(edgelist, Rᵢⱼ, M)


## EPIDEMIC PARAMETERS HUMAN BEHAVIOUR
# Average number of contacts per strata
kᵍ = Float64.(epiparams_dict["kᵍ"])
# Average number of contacts at home per strata
kᵍ_h = Float64.(epiparams_dict["kᵍ_h"])
# Average number of contacts at work per strata
kᵍ_w = Float64.(epiparams_dict["kᵍ_w"])
# Degree of mobility per strata
pᵍ = Float64.(epiparams_dict["pᵍ"])
# Density factor
ξ = epiparams_dict["σ"]
# Average household size
σ = epiparams_dict["σ"]
# Check network structure and self-loop correction


## EPIDEMIC PARAMETERS TRANSITION RATES
# Scaling of the asymptomatic infectivity
scale_β = epiparams_dict["scale_β"]
# Infectivity of Symptomatic
βᴵ = epiparams_dict["βᴵ"]
# Infectivity of Asymptomatic
βᴬ = scale_β * βᴵ
# Exposed rate
ηᵍ = Float64.(epiparams_dict["ηᵍ"])
# Asymptomatic rate
αᵍ = Float64.(epiparams_dict["αᵍ"])
# Infectious rate
μᵍ = Float64.(epiparams_dict["μᵍ"])


## EPIDEMIC PARAMETERS TRANSITION RATES VACCINATION
# Direct death probability
θᵍ = Float64.(reduce(hcat, [epiparams_dict["θᵍ"], epiparams_dict["θᵍ"] * epiparams_dict["risk_reduction_dd"]]) )
# Hospitalization probability
γᵍ = Float64.(reduce(hcat, [epiparams_dict["γᵍ"], epiparams_dict["γᵍ"] * epiparams_dict["risk_reduction_h"]]) )
# Fatality probability in ICU
ωᵍ = Float64.(reduce(hcat, [epiparams_dict["ωᵍ"], epiparams_dict["ωᵍ"] * epiparams_dict["risk_reduction_d"]]) )
# Pre-deceased rate
ζᵍ = Float64.(epiparams_dict["ζᵍ"])
# Pre-hospitalized in ICU rate
λᵍ = Float64.(epiparams_dict["λᵍ"])
# Death rate in ICU
ψᵍ = Float64.(epiparams_dict["ψᵍ"])
# ICU discharge rate
χᵍ = Float64.(epiparams_dict["χᵍ"])

# Waning immunity rate 
Λ = epiparams_dict["Λ"] 
# Reinfection rate
Γ = epiparams_dict["Γ"] 
# Relative risk reduction of the probability of infection
rᵥ = Float64.(epiparams_dict["rᵥ"])
# Relative risk reduction of the probability of transmission
kᵥ = Float64.(epiparams_dict["kᵥ"])

2-element Vector{Float64}:
 0.0
 0.4

In [8]:
println("M = ", M)
println("G = ", G)
println("T = ", T)
println("V = ", V)
println("num_compartments : ", num_compartments)

M = 2850
G = 3
T = 30
V = 2
num_compartments : 10


In [9]:
#########################################################
# Vaccination parameters
#########################################################

# vaccionation dates
start_vacc = vacparams_dict["start_vacc"]
dur_vacc   = vacparams_dict["dur_vacc"]
end_vacc   = start_vacc + dur_vacc

# total vaccinations per age strata
ϵᵍ = vacparams_dict["ϵᵍ"] * round( total_population * vacparams_dict["percentage_of_vacc_per_day"] )

tᵛs = [start_vacc, end_vacc, T]
ϵᵍs = ϵᵍ .* [0  Int(vacparams_dict["are_there_vaccines"])  0] 

3×3 Matrix{Float64}:
 0.0  23189.1        0.0
 0.0  92756.4        0.0
 0.0      1.15946e5  0.0

In [10]:
#########################################################
# Containement measures
#########################################################

# Mobility reduction
κ₀_df = CSV.read(joinpath(data_path, config["data"]["kappa0_filename"]), DataFrame);

# syncronize containment measures with simulation
κ₀_df.time = map(x -> (x .- first_day).value + 1, κ₀_df.date)

# Timesteps when the containment measures will be applied
tᶜs = Int64.(npiparams_dict["tᶜs"])

# Array of level of confinement
# κ₀s = κ₀_df.reduction[:]
κ₀s = Float64.(npiparams_dict["κ₀s"])
# Array of premeabilities of confined households
ϕs = Float64.(npiparams_dict["ϕs"])
# Array of social distancing measures
δs = Float64.(npiparams_dict["δs"])

1-element Vector{Float64}:
 0.8

INITIALIZATION OF THE EPIDEMICS

In [11]:
# Load initial conditions
if initial_compartments_path != nothing
    # use initial compartments matrix to initialize simulations
    initial_compartments = h5open(initial_compartments_path, "r") do file
        read(file, "compartments")
    end
else
    # use seeds to initialize simulations
    initial_compartments = nothing
end

# Initial seeds (intial condition at the begining of the pandemic)
conditions₀ = CSV.read(A0_instance_filename, DataFrame)

####################################
## INITIALIZATION OF THE EPIDEMICS
####################################

Sᵛ₀ = zeros(Float64,G, M)
E₀ = zeros(Float64, G, M) # Initial number of exposed individuals
A₀ = zeros(Float64, G, M) # Initial number of infectious asymptomatic individuals
I₀ = zeros(Float64, G, M)
H₀ = zeros(Float64, G, M)
R₀ = zeros(Float64, G, M)

if A0_instance_filename !== nothing

    # Initial number of infectious asymptomatic individuals
    # use seeds to initialize simulations
    conditions₀ = CSV.read(A0_instance_filename, DataFrame)        
    
    A₀[1, Int.(conditions₀[:,"idx"])] .= 0.12 .* conditions₀[:,"seed"]
    A₀[2, Int.(conditions₀[:,"idx"])] .= 0.16 .* conditions₀[:,"seed"]
    A₀[3, Int.(conditions₀[:,"idx"])] .= 0.72 .* conditions₀[:,"seed"]

    
else
    E₀ = nᵢᵍ / total_population * 1000
    A₀ = nᵢᵍ / total_population * 1000
    I₀ = nᵢᵍ / total_population * 1000    
end

println("Total population = ", total_population)

Total population = 4.6378115e7


SETTING UP SIMULATION VARIABLES

In [12]:
# Parameters to simulate
paramsDF = CSV.read(joinpath(instance_path, fitting_params["parameters_filename"]), DataFrame)

total_simulations = length(paramsDF.id)
println("total_simulations : ", total_simulations)

compartments = zeros(Float64, G, M, T, V, num_compartments, total_simulations);
size(compartments)



total_simulations : 1


(3, 2850, 30, 2, 10, 1)

SETTING UP THE THREADING VARIABLES

In [13]:
nThreads = nthreads();
# nThreads = 1;
println("nThreads: ", nThreads)
flush(stdout)

# Circular queues to optimize and reuse the population and epidemic structures
populations = CircularDeque{Population_Params}(nThreads)
epi_params = CircularDeque{Epidemic_Params}(nThreads)

# Populate the circular deque
for t in 1:nThreads
    push!(populations, Population_Params(G, M, nᵢᵍ, kᵍ, kᵍ_h, kᵍ_w, C, pᵍ, edgelist, Rᵢⱼ, sᵢ, ξ, σ))
    push!(epi_params, Epidemic_Params(βᴵ,  βᴬ, ηᵍ, αᵍ, μᵍ, θᵍ, γᵍ, ζᵍ, λᵍ, ωᵍ, ψᵍ, χᵍ,  Λ, Γ, rᵥ, kᵥ, G, M, T, V))
end

nThreads: 1


In [14]:
# Patch to CCAA mapping matrix
PatchToCCAA = npzread(joinpath(data_path, fitting_params["patch_to_ccaa_filename"]))
n_ccaa = size(PatchToCCAA)[1]
n_patches = size(PatchToCCAA)[2]

2850

RUN THE SIMULATION

In [15]:
# Setup progress bar
p = Progress(total_simulations)

# Circular Deque Lock
lockS = SpinLock()

# Run the simulation for all the parameters
@threads for indx_id in 1:total_simulations
    
    # println("Thread: ", threadid(), " row: ", indx_id)

    # Lock the data structure
    lock(lockS)

    # Recover population and epidemic parameters
    population = pop!(populations)
    epi_param = pop!(epi_params)

    # Unlock data structure
    unlock(lockS)

    # Run simu
    run_simu_params!(epi_param,
                     population,
                     paramsDF,
                     indx_id,
                     E₀,
                     A₀,
                     I₀,
                     H₀,
                     R₀,
                     Sᵛ₀,
                     compartments)

    # Lock the data structure
    lock(lockS)

    # Free population and epidemic parameters
    push!(populations, population)
    push!(epi_params, epi_param)

    # Unlock data structure
    unlock(lockS)

    # Update progress bar
    next!(p)
    
end

Time: 1, players: 46378115.00000, icus: 0.00, deaths: 0.00, vaccine_check: 0.000
Time: 2, players: 46378115.00000, icus: 0.00, deaths: 0.00, vaccine_check: 0.000


Time: 3, players: 46378115.00000, icus: 0.00, deaths: 0.00, vaccine_check: 0.000
Time: 4, players: 46378115.00000, icus: 1.66, deaths: 0.00, vaccine_check: 0.000


Time: 5, players: 46378115.00000, icus: 2.65, deaths: 0.07, vaccine_check: 0.000
Time: 6, players: 46378115.00000, icus: 3.49, deaths: 0.17, vaccine_check: 0.000


Time: 7, players: 46378115.00000, icus: 4.24, deaths: 0.31, vaccine_check: 0.000
Time: 8, players: 46378115.00000, icus: 5.04, deaths: 0.45, vaccine_check: 0.000


Time: 9, players: 46378115.00000, icus: 5.98, deaths: 0.62, vaccine_check: 0.000
Time: 10, players: 46378115.00000, icus: 7.19, deaths: 0.80, vaccine_check: 0.000


...

Time: 21, players: 46378115.00000, icus: 174.08, deaths: 11.54, vaccine_check: 0.000
Time: 22, players: 46378115.00000, icus: 242.66, deaths: 15.69, vaccine_check: 0.000


Time: 23, players: 46378115.00000, icus: 338.39, deaths: 21.48, vaccine_check: 0.000
Time: 24, players: 46378115.00000, icus: 471.92, deaths: 29.58, vaccine_check: 0.000


Time: 25, players: 46378115.00000, icus: 658.01, deaths: 40.88, vaccine_check: 0.000
Time: 26, players: 46378115.00000, icus: 917.10, deaths: 56.66, vaccine_check: 0.000


Time: 27, players: 46378115.00000, icus: 1277.47, deaths: 78.67, vaccine_check: 0.000
Time: 28, players: 46378115.00000, icus: 1778.11, deaths: 109.37, vaccine_check: 0.000


Time: 29, players: 46378115.00000, icus: 2472.58, deaths: 152.12, vaccine_check: 0.000
Time: 30, players: 46378115.00000, icus: 3434.10, deaths: 211.62, vaccine_check: 0.000


STORING THE RESULTS

In [16]:
#This creates a new NetCDF file
#The mode "c" stands for creating a new file

#NetCDF_file = Dataset("test.nc","c")

#Define the dimension
#defDim(NetCDF_file, "G", G)
#defDim(NetCDF_file, "M", M)
#defDim(NetCDF_file, "T", T)
#defDim(NetCDF_file, "V", V)
#defDim(NetCDF_file, "num_comp", num_compartments)

#Define the variables 
#v = defVar(NetCDF_file,"population", Float64, ("G", "M", "T", "V", "num_comp"))
#v1 = defVar(NetCDF_file,"G",Float64)
#v2 = defVar(NetCDF_file,)
#v3 =
#v4 =
#v5 = 
#G = dfVar

#v[:,:,:,:,:] = compartments

#close(NetCDF_file)

In [17]:
G_dir = ["C", "A", "I"]


3-element Vector{String}:
 "C"
 "A"
 "I"

In [22]:
#Define the coordinates
G_dir = collect(1:G)
#G_dir = ["C", "A", "I"]
M_dir = collect(1:M)
T_dir = collect(1:T)
V_dir = collect(1:V)
ncomp_dir = collect(1:num_compartments)

#Atributes
#G_atts = Dict("Age strata")
#M_atts = Dict("Number of patches")
#T_atts = Dict("number of time steps")
#V_atts = Dict("Vaccination")
#ncomp_atts = Dict("Number of compartments")

file = "tes1.nc"
isfile(file) && rm(file)
#nccreate(file, "data", "G", G_dir, G_atts, "M", M_dir, M_atts, "T", T_dir, T_atts, "V", V_dir, V_atts, "n_comp", ncomp_dir, ncomp_atts )
nccreate(file, "data", "G", G_dir, "M", M_dir, "T", T_dir, "V", V_dir, "n_comp", ncomp_dir)

ncwrite(compartments[:,:,:,:,:,1], file, "data")

In [19]:
println("Storing results to: $output_path")

if !isdir(output_path)
  mkpath(output_path)
end



# store compartments
if export_compartments
    println("Storing compartments")
    for i in 1:total_simulations

        id = paramsDF[i, "id"]

        if export_compartments_time_t != nothing
            filename = joinpath(output_path, "compartments_$(export_compartments_date)_$id.h5")
            # filename = joinpath(output_path, "compartments_$(export_compartments_date):$(export_compartments_time_t)_$id.h5")
            h5open(filename, "w") do file
                write(file, "compartments", compartments[:,:,export_compartments_time_t,:,:,i])
            end
        end

        if export_compartments_full
            filename = joinpath(output_path, "compartments_full_$id.h5")
            h5open(filename, "w") do file
                write(file, "compartments", compartments[:,:,:,:,:,i])
            end
        end
    end
end

Storing results to: ../test/output
Storing compartments


In [20]:
#push!(epi_params, epi_param)
#epi_param = pop!(epi_params)
fieldnames(typeof(epi_params))
#epi_param."βᴵ"

(:buffer, :capacity, :n, :first, :last)